In [1]:
from __future__ import print_function
import cv2
import time
from imutils.video import FPS, WebcamVideoStream
import argparse

import warnings
warnings.filterwarnings("ignore")

from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, LearningRateScheduler
from keras.callbacks import Callback
from keras import backend as K 
from keras.models import load_model
from math import ceil 
import numpy as np 
from termcolor import colored

from mn_model import mn_model
from face_generator import BatchGenerator
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2

# training parameters
from keras import backend as K
import scipy.misc as sm

Using TensorFlow backend.


In [2]:
COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
FONT = cv2.FONT_HERSHEY_SIMPLEX

img_height =512
img_width = 512

img_channels = 3

n_classes =2 
class_names = ["background","face"]

scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # anchorboxes
aspect_ratios = [[0.5, 1.0, 2.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [0.5, 1.0, 2.0],
                 [0.5, 1.0, 2.0]] # The anchor box aspect ratios used in the original SSD300
two_boxes_for_ar1 = True
limit_boxes = True # Whether or not you want to limit the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are scaled as in the original implementation
coords = 'centroids' # Whether the box coordinates to be used as targets for the model should be in the 'centroids' or 'minmax' format, see documentation
normalize_coords = True

det_model_path = "./models/"

In [3]:
def cv2_demo(model):
    def predict(frame):
        height, width = frame.shape[:2]

        _CONF = 0.60 
        _IOU = 0.15

        frame_scaled = np.zeros((img_height, img_width, img_channels))

        for idx in range(img_channels):
            frame_scaled[:, :, idx] = cv2.resize(frame[:, :, idx], (img_height, img_width), interpolation=cv2.INTER_CUBIC)
    
        img = np.expand_dims(frame_scaled, axis=0)

        y_pred = model.predict(img)


        result = decode_y2(y_pred,
                                confidence_thresh=_CONF,
                                iou_threshold=_IOU,
                                top_k='all',
                                input_coords=coords,
                                normalize_coords=normalize_coords,
                                img_height=img_height,
                                img_width=img_width)[0]
        
        np.set_printoptions(suppress=True)        
                
        # scale each detection back up to the image
        scale = np.array([width/img_width, width/img_width, height/img_height, height/img_height])

        for i in range(len(result)):
            if result[i][1] < 0.6:
                print('conf < 0.6')
            pt = result[i][2:]
            pt *= scale
            pt = pt.astype(int)
            cv2.rectangle(frame, (pt[0], pt[2]), (pt[1], pt[3]), COLORS[i % 3], 2)
#             cv2.putText(frame, 'person', (pt[0], pt[2]), FONT, 2, (255, 255, 255), 2, cv2.LINE_AA)
        return frame

    # start video stream thread, allow buffer to fill
    print("[INFO] starting threaded video stream...")
    stream = WebcamVideoStream(src=0).start()  # default camera
    time.sleep(1.0)
    # start fps timer
    # loop over frames from the video file stream
    while True:
        # grab next frame
        frame = stream.read()
        key = cv2.waitKey(1) & 0xFF

        # update FPS counter
        fps.update()
        frame = predict(frame)

        # keybindings for display
        if key == ord('p'):  # pause
            while True:
                key2 = cv2.waitKey(1) or 0xff
                cv2.imshow('frame', frame)
                if key2 == ord('p'):  # resume
                    break
        cv2.imshow('frame', frame)
        if key == 27:  # exit
            break
    stream.stop()    

In [4]:
if __name__ == '__main__':
    model, model_layer, img_input, predictor_sizes = mn_model(image_size=(img_height, img_width, img_channels), 
                                                                      n_classes = n_classes,
                                                                      min_scale = None, 
                                                                      max_scale = None, 
                                                                      scales = scales, 
                                                                      aspect_ratios_global = None, 
                                                                      aspect_ratios_per_layer = aspect_ratios, 
                                                                      two_boxes_for_ar1= two_boxes_for_ar1, 
                                                                      limit_boxes=limit_boxes, 
                                                                      variances= variances, 
                                                                      coords=coords, 
                                                                      normalize_coords=normalize_coords)
    
    model_path = './models/'
    model_name = 'ssd_mobilenet_face_epoch_25_loss0.0916.h5'

    model.load_weights(model_path + model_name,  by_name= True)

    print (colored('weights %s loaded' % (model_path + model_name), 'green'))

    fps = FPS().start()
    cv2_demo(model)
    # stop the timer and display FPS information
    fps.stop()

    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    # cleanup
    cv2.destroyAllWindows()

====> Model Specific data
====> Height, Width, Channels : 512 512 3
weights ./models/ssd_mobilenet_face_epoch_25_loss0.0916.h5 loaded
[INFO] starting threaded video stream...
[INFO] elasped time: 14.61
[INFO] approx. FPS: 0.82
